## lib

In [1]:
from __future__ import division, print_function
%matplotlib inline
from importlib import reload  # Python 3
import utils; reload(utils)
from utils import *

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
Using TensorFlow backend.


## parameters

In [2]:
#patch
from keras import backend as K
K.set_image_dim_ordering('th')

In [3]:
path = "convbattle2/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

#batch_size=1
batch_size=64

## custom function

In [4]:
def get_batches_channelfirst(dirname, gen=image.ImageDataGenerator(data_format="channels_first"), shuffle=True, batch_size=4, class_mode='categorical',
                target_size=(224,224)):
    return gen.flow_from_directory(dirname, target_size=target_size,
            class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [5]:
# SINCE KERAS MAKES USE OF INVERTED DROPOUT WE "NEUTRALIZE" proc_wgts(layer):
#def proc_wgts(layer): return [o for o in layer.get_weights()]

In [6]:
# SINCE KERAS MAKES USE OF INVERTED DROPOUT WE "NEUTRALIZE" proc_wgts(layer):
#def proc_wgts(layer, prev_p, new_p):
def proc_wgts(layer, scal): return [o*scal for o in layer.get_weights()]

"""
def proc_wgts should be

def proc_wgts(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]
"""

In [7]:
def get_fc_model():
    model = Sequential([
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.1),
        Dense(4096, activation='relu'),
        Dropout(0.2),
        Dense(2, activation='softmax')
        ])

    #transfer fc_layers weights to returned model.layers weights
    for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2, 1))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        #Dense(1000, activation='softmax')
        Dense(2, activation='softmax')
        ]

## model initialisation

In [9]:
model = vgg_ft(2)
model.load_weights(model_path+'finetune.lesson2fromscratch.startlesson3.h5')

In [10]:
#evaluate finetune model
val_batches = get_batches(path+'valid', shuffle=False, batch_size=1)
val_labels = onehot(val_batches.classes)

val_data = np.rollaxis(load_array(model_path+'valid_data.bc'), 3, 1)

model.evaluate(val_data, val_labels)

Found 551 images belonging to 2 classes.
551/551 [==============================] - 15s    


[0.5055508043329352, 0.87658802177858441]

## data preparations

In [11]:
# /!\ shuffle batches is at FALSE :-0
batches = get_batches_channelfirst(path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches_channelfirst(path+'valid', shuffle=False, batch_size=batch_size)
steps_per_epoch = int(np.ceil(batches.samples/batch_size))
validation_steps = int(np.ceil(val_batches.samples/batch_size))

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 2213 images belonging to 2 classes.
Found 551 images belonging to 2 classes.


In [12]:
# highly computationnal opérations
# activate only if you never computed nor saved theses files
generate_feature = False

if generate_feature:
    print("generating features")
    val_features = conv_model.predict_generator(val_batches, validation_steps)
    trn_features = conv_model.predict_generator(batches, steps_per_epoch)
    save_array(model_path + 'train_convlayer_features.bc', trn_features)
    save_array(model_path + 'valid_convlayer_features.bc', val_features)
else:
    print("loading features")
    trn_features = load_array(model_path+'train_convlayer_features.bc')
    val_features = load_array(model_path+'valid_convlayer_features.bc')
    
print (trn_features.shape)
print (val_features.shape)

loading features
(2213, 512, 14, 14)
(551, 512, 14, 14)


## model customization

In [13]:
layers = model.layers

last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]
#print ("last convolutionnal layer id : " + str(last_conv_idx))

conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)

fc_layers = layers[last_conv_idx+1:]

In [14]:
# Such a finely tuned model needs to be updated very slowly!
opt = RMSprop(lr=0.00001, rho=0.7)

In [15]:
fc_model = get_fc_model()
#get_fc_model include compiling op

In [ ]:
fc_model.summary()

## model fitting

In [16]:
fc_model.fit(trn_features, trn_labels, epochs=1, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 2213 samples, validate on 551 samples
Epoch 1/1
2213/2213 [==============================] - 5s - loss: 0.0733 - acc: 0.9806 - val_loss: 0.5099 - val_acc: 0.8875


In [22]:
#fc_model.save_weights(model_path+'dropout.h5')

In [23]:
#if needed for restart
#fc_model.load_weights(model_path+'dropout.h5')

## Reduce overfitting > adding data augmentation

In [17]:
gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.2, 
       height_shift_range=0.2, shear_range=0.15, zoom_range=0.2, 
       channel_shift_range=15., horizontal_flip=True)

In [18]:
batches = get_batches_channelfirst(path+'train', gen, batch_size=batch_size)
# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches_channelfirst(path+'valid', shuffle=False, batch_size=batch_size)

steps_per_epoch = int(np.ceil(batches.samples/batch_size))
validation_steps = int(np.ceil(val_batches.samples/batch_size))

Found 2213 images belonging to 2 classes.
Found 551 images belonging to 2 classes.


In [19]:
#in order to allow data to flow through all the conv layers and our new dense layers, 
#we attach our fully connected model to the convolutional model--after ensuring that the convolutional layers 
#are not trainable:

#if needed for restart
#conv_layers = layers[:last_conv_idx+1]
#conv_model = Sequential(conv_layers)

# Look how easy it is to connect two models together!
for layer in conv_model.layers: layer.trainable = False
conv_model.add(fc_model)

In [20]:
#conv_model.summary()

Now we can compile, train, and save our model as usual - note that we use *fit_generator()* since we want to pull random images from the directories on every batch.

In [21]:
opt = RMSprop(lr=0.00000003, rho=0.7)
conv_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
#conv_model.save_weights(model_path + 'aug1.epoch00.h5')

In [25]:
conv_model.fit_generator(batches, steps_per_epoch=steps_per_epoch, epochs=1, 
                        validation_data=val_batches, validation_steps=validation_steps)
conv_model.save_weights(model_path + 'aug1.epoch01.h5')

Epoch 1/1
35/35 [==============================] - 65s - loss: 0.6095 - acc: 0.8690 - val_loss: 0.5006 - val_acc: 0.8875


In [26]:
conv_model.fit_generator(batches, steps_per_epoch=steps_per_epoch, epochs=1, 
                        validation_data=val_batches, validation_steps=validation_steps)
conv_model.save_weights(model_path + 'aug1.epoch02.h5')

Epoch 1/1
35/35 [==============================] - 53s - loss: 0.5386 - acc: 0.8708 - val_loss: 0.4931 - val_acc: 0.8929


In [27]:
conv_model.fit_generator(batches, steps_per_epoch=steps_per_epoch, epochs=1, 
                        validation_data=val_batches, validation_steps=validation_steps)
conv_model.save_weights(model_path + 'aug1.epoch03.h5')

Epoch 1/1
35/35 [==============================] - 53s - loss: 0.5070 - acc: 0.8771 - val_loss: 0.4876 - val_acc: 0.8911


In [28]:
conv_model.fit_generator(batches, steps_per_epoch=steps_per_epoch, epochs=1, 
                        validation_data=val_batches, validation_steps=validation_steps)
conv_model.save_weights(model_path + 'aug1.epoch04.h5')

Epoch 1/1
35/35 [==============================] - 53s - loss: 0.6147 - acc: 0.8671 - val_loss: 0.4825 - val_acc: 0.8911


## Reduce overfitting > adding batch normalization

**all modern networks should use batchnorm, or something equivalent**. There are two reasons for this:
1. Adding batchnorm to a model can result in **10x or more improvements in training speed**
2. Because normalization greatly reduces the ability of a small number of outlying inputs to over-influence the training, it also tends to **reduce overfitting**.

In [29]:
conv_model.load_weights(model_path + 'aug1.epoch03.h5')

In [30]:
conv_layers[-1].output_shape[1:]

(512, 14, 14)

In [31]:
bn_model = Sequential(get_bn_layers(0.2))

In [32]:
for l in bn_model.layers: 
    if type(l)==Dense: l.set_weights(proc_wgts(l, 1))

In [33]:
bn_model.pop()
for layer in bn_model.layers: layer.trainable=False

In [34]:
bn_model.add(Dense(2,activation='softmax'))

In [35]:
bn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling2d_7 (MaxPooling2 (None, 512, 7, 7)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dropout_5 (Dropout)          (None, 4096)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 4096)              16384     
_________________________________________________________________
dense_9 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dropout_6 (Dropout)          (None, 4096)              0         
__________

In [36]:
bn_model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [37]:
bn_model.fit(trn_features, trn_labels, epochs=3, validation_data=(val_features, val_labels))

Train on 2213 samples, validate on 551 samples
Epoch 1/3
2213/2213 [==============================] - 1s - loss: 0.6971 - acc: 0.6801 - val_loss: 0.7887 - val_acc: 0.6951
Epoch 2/3
2213/2213 [==============================] - 1s - loss: 0.4292 - acc: 0.8229 - val_loss: 0.4423 - val_acc: 0.8076
Epoch 3/3
2213/2213 [==============================] - 1s - loss: 0.3802 - acc: 0.8405 - val_loss: 0.3996 - val_acc: 0.8312


In [38]:
bn_model.save_weights(model_path+'bn.dropout20.epoch03.h5')

In [39]:
bn_model.fit(trn_features, trn_labels, epochs=2, validation_data=(val_features, val_labels))

Train on 2213 samples, validate on 551 samples
Epoch 1/2
2213/2213 [==============================] - 1s - loss: 0.3044 - acc: 0.8717 - val_loss: 0.3964 - val_acc: 0.8385
Epoch 2/2
2213/2213 [==============================] - 1s - loss: 0.2926 - acc: 0.8735 - val_loss: 0.3935 - val_acc: 0.8475


In [40]:
bn_model.fit(trn_features, trn_labels, epochs=2, validation_data=(val_features, val_labels))

Train on 2213 samples, validate on 551 samples
Epoch 1/2
2213/2213 [==============================] - 1s - loss: 0.2786 - acc: 0.8848 - val_loss: 0.4163 - val_acc: 0.8330
Epoch 2/2
2213/2213 [==============================] - 1s - loss: 0.2849 - acc: 0.8897 - val_loss: 0.4053 - val_acc: 0.8439


In [ ]:
#reload previous training
#bn_model.load_weights(model_path+'bn.dropout20.epoch05.h5')

In [41]:
bn_layers = get_bn_layers(0.6)
bn_layers.pop()
bn_layers.append(Dense(2,activation='softmax'))

In [42]:
final_model = Sequential(conv_layers)
for layer in final_model.layers: layer.trainable = False
for layer in bn_layers: final_model.add(layer)

In [43]:
for l1,l2 in zip(bn_model.layers, bn_layers):
    l2.set_weights(l1.get_weights())

In [44]:
final_model.compile(optimizer=Adam(), 
                    loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
final_model.fit_generator(batches, steps_per_epoch=steps_per_epoch, epochs=1, 
                        validation_data=val_batches, validation_steps=validation_steps)

Epoch 1/1
35/35 [==============================] - 57s - loss: 1.5154 - acc: 0.7494 - val_loss: 4.0882 - val_acc: 0.6842


In [46]:
final_model.fit_generator(batches, steps_per_epoch=steps_per_epoch, epochs=1, 
                        validation_data=val_batches, validation_steps=validation_steps)

Epoch 1/1
35/35 [==============================] - 56s - loss: 0.9484 - acc: 0.8100 - val_loss: 2.6715 - val_acc: 0.7296


In [ ]:
#final_model.save_weights(model_path + 'final1.h5')

In [47]:
final_model.optimizer.lr=0.001

In [48]:
final_model.fit_generator(batches, steps_per_epoch=steps_per_epoch, epochs=2, 
                        validation_data=val_batches, validation_steps=validation_steps)

Epoch 1/2
35/35 [==============================] - 56s - loss: 0.5424 - acc: 0.8360 - val_loss: 0.7430 - val_acc: 0.8711
Epoch 2/2
35/35 [==============================] - 56s - loss: 0.4470 - acc: 0.8573 - val_loss: 0.6175 - val_acc: 0.8603


In [49]:
bn_model.save_weights(model_path + 'final3.h5')

In [50]:
final_model.fit_generator(batches, steps_per_epoch=steps_per_epoch, epochs=2, 
                        validation_data=val_batches, validation_steps=validation_steps)

Epoch 1/2
35/35 [==============================] - 57s - loss: 0.4422 - acc: 0.8446 - val_loss: 0.7405 - val_acc: 0.8348
Epoch 2/2
35/35 [==============================] - 55s - loss: 0.3221 - acc: 0.8916 - val_loss: 0.5672 - val_acc: 0.8457


In [54]:
bn_model.load_weights(model_path + 'final3.h5')

In [55]:
final_model.optimizer.lr=0.00003

In [56]:
final_model.fit_generator(batches, steps_per_epoch=steps_per_epoch, epochs=1, 
                        validation_data=val_batches, validation_steps=validation_steps)

Epoch 1/1
35/35 [==============================] - 56s - loss: 0.3191 - acc: 0.8924 - val_loss: 0.4885 - val_acc: 0.8603


In [ ]:
#bn_model.save_weights(model_path + 'final4.h5')